In [35]:
import os
import json
filename_list = os.listdir('../data')
filename_list = sorted(filename_list, key=lambda x: (len(x), x))
tag_2 = filename_list[0]
filename_list = filename_list[1:]
print(tag_2, filename_list, len(filename_list))
rel_list = []
with open('../data/' + tag_2, 'r') as f:
    lines = f.readlines()
    lines = list(map(lambda x: x.replace('\n', ''), lines))
    f_list = list(map(lambda x: x.split(' ')[0], lines))
    rel_list = list(map(lambda x: x.split(' ')[1], lines))
    temp = sorted(zip(f_list, rel_list), key=lambda x: (len(x[0]), x[0]))
    f_list, rel_list = zip(*temp)
    f_list = list(f_list)
    rel_list = list(rel_list)
    print(f_list)
    print(rel_list)

tag_2.txt ['1_data1.txt', '2_data1.txt', '3_data1.txt', '4_data1.txt', '5_data1.txt', '6_data1.txt', '7_data1.txt', '8_data1.txt', '9_data1.txt', '10_data1.txt', '11_data1.txt', '12_data1.txt', '13_data1.txt', '14_data1.txt', '15_data1.txt', '16_data1.txt', '17_data1.txt', '18_data1.txt', '19_data1.txt', '20_data1.txt', '21_data1.txt', '22_data1.txt', '23_data1.txt', '24_data1.txt', '25_data1.txt', '26_data1.txt', '27_data1.txt', '28_data1.txt', '29_data1.txt', '30_data1.txt', '32_data1.txt', '34_data1.txt', '35_data1.txt', '36_data1.txt', '37_data1.txt', '38_data1.txt', '39_data1.txt', '40_data1.txt', '41_data1.txt', '42_data1.txt', '43_data1.txt', '44_data1.txt', '45_data1.txt', '46_data1.txt', '47_data1.txt', '48_data1.txt', '49_data1.txt', '50_data1.txt', '51_data1.txt', '52_data1.txt', '53_data1.txt', '54_data1.txt', '55_data1.txt', '56_data1.txt', '57_data1.txt', '58_data1.txt', '59_data1.txt', '60_data1.txt', '61_data1.txt', '62_data1.txt', '63_data1.txt', '64_data1.txt', '65_da

In [47]:
dataset_list = []
for filename, rel in zip(filename_list, rel_list):
    dataset = {}
    with open('../data/' + filename, 'r') as f:
        lines = f.readlines()
        filename = filename.replace('1.txt', '.txt')
        dataset['filename'] = filename
        lines = list(map(lambda x: x.replace('\n', ''), lines))
        data = list(map(lambda x: x.split(' ')[0], lines))
        label = list(map(lambda x: x.split(' ')[1], lines))
        text = ' '.join(data)
        dataset['token'] = data
        dataset['bio_label'] = label
        dataset['text'] = text
        dataset['relation'] = rel
        dataset_list.append(dataset)

In [52]:
dataset_list[0]['relation']

'110000300001100311010'

In [53]:
dataset_list_json = json.dumps(dataset_list)
with open('datasets.json', 'w') as f:
    f.write(dataset_list_json)